In [ ]:
import torch
from torch.utils.data import DataLoader
import pytorch3d
import sys

import pandas as pd 

from shapeaxi import saxi_nets
from shapeaxi import saxi_dataset
from shapeaxi import saxi_dataset
from shapeaxi import saxi_transforms

import plotly.express as px

from pytorch3d.structures import Meshes d
from pytorch3d.renderer import (
        FoVPerspectiveCameras, look_at_view_transform, look_at_rotation, 
        RasterizationSettings, MeshRenderer, MeshRasterizer, MeshRendererWithFragments, BlendParams,
        SoftSilhouetteShader, HardPhongShader, SoftPhongShader, AmbientLights, PointLights, TexturesUV, TexturesVertex, TexturesAtlas
)

In [ ]:
# model = saxi_nets.SaxiRingClassification(subdivision_level=3, out_classes=3, radius=1.1, hidden_dim=512, out_size=128, dropout_lvl=0.2, image_size=224, base_encoder="ViT", base_encoder_params="in_channels=4, img_size=(224,224), patch_size=(16,16),spatial_dims=2")
model = saxi_nets.SaxiRingClassification(subdivision_level=1, out_classes=3, radius=1.05, hidden_dim=512, out_size=128, dropout_lvl=0.2, image_size=224, base_encoder="resnet18", base_encoder_params="pretrained=False,spatial_dims=2,n_input_channels=4,num_classes=512")
model = model.to('cuda').eval()

In [ ]:
ds = saxi_dataset.SaxiDataset(pd.read_csv('/CMF/data/lumargot/DCBIA/Airway_Obst_Classif_Sample/airway_4classes_train.csv'), transform=saxi_transforms.EvalTransform())

In [ ]:
V, F, CN = ds[0]

In [ ]:

def render(self, V, F, CN):
    # Render the input surface mesh to an image
    textures = TexturesVertex(verts_features=CN)
    meshes = Meshes(verts=V, faces=F, textures=textures)
    X = []
    PF = []

    for camera_position in self.ico_verts:
        camera_position = camera_position.unsqueeze(0)
        camera_position = camera_position.to(self.device)
        R = look_at_rotation(camera_position, device=self.device)  # (1, 3, 3)
        T = -torch.bmm(R.transpose(1, 2), camera_position[:,:,None])[:, :, 0]   # (1, 3)
        images = self.renderer(meshes_world=meshes.clone(), R=R, T=T)
        fragments = self.renderer.rasterizer(meshes.clone())
        
        pix_to_face = fragments.pix_to_face
        zbuf = fragments.zbuf

        v = V[:,F[:,pix_to_face][:,:,:,:,:,0]].squeeze(dim=5).squeeze(dim=1).squeeze(dim=1)
        
        z_buf_n = torch.square(v - camera_position).sum(dim=-1).unsqueeze(-1)

        images = torch.cat([images[:,:,:,0:3], z_buf_n], dim=-1)
        
        images = images.permute(0,3,1,2)
        pix_to_face = pix_to_face.permute(0,3,1,2)
        
        X.append(images.unsqueeze(1))
        PF.append(pix_to_face.unsqueeze(1))
    
    X = torch.cat(X, dim=1)
    PF = torch.cat(PF, dim=1)

    return X, PF

X, PF = render(model, V.unsqueeze(0).cuda(), F.unsqueeze(0).cuda(), CN.unsqueeze(0).cuda())

px.imshow(X[0,:,3:].permute(0,2,3,1).squeeze().cpu().numpy(), animation_frame=0)

# px.imshow(, animation_frame=0)

In [ ]:
torch.cuda.empty_cache()